In [1]:
# Ensure CUDA and cuDNN are installed
!nvcc --version
!nvidia-smi
# Install the required dependencies for building TensorFlow with TensorRT support
!sudo apt-get update
!sudo apt-get install -y libnvinfer8 libnvinfer-dev libnvinfer-plugin8
# (Install other necessary packages as mentioned in TensorFlow documentation)
# Clone the TensorFlow repository and checkout the desired branch
!git clone https://github.com/tensorflow/tensorflow.git
%cd tensorflow
!git checkout r2.10 # Check the TensorFlow-TensorRT compatibility matrix for the correct branch.
# Configure TensorFlow build with TensorRT enabled
# ./configure
# (During configuration, enable TensorRT support when prompted)
# If you are using a virtual environment, activate it before building TensorFlow.
# Build and install TensorFlow
!bazel build --config=cuda --config=monolithic ... (Specify the build target with TensorRT support)
!bazel install ... (Install the built TensorFlow package)
# After successful installation, restart the runtime to ensure the new TensorFlow installation is used.

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2023 NVIDIA Corporation
Built on Tue_Aug_15_22:02:13_PDT_2023
Cuda compilation tools, release 12.2, V12.2.140
Build cuda_12.2.r12.2/compiler.33191640_0
Fri Dec 27 06:08:47 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  Tesla T4                       Off | 00000000:00:04.0 Off |                    0 |
| N/A   38C    P8               9W /  7

In [2]:
!nvidia-smi

Fri Dec 27 06:08:59 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  Tesla T4                       Off | 00000000:00:04.0 Off |                    0 |
| N/A   38C    P8               9W /  70W |      0MiB / 15360MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

In [3]:
!pip install tf2onnx

In [4]:
%cd ~

/root


In [5]:
import tensorflow as tf
from tensorflow.python.compiler.tensorrt import trt_convert as trt
import numpy as np
import os

In [6]:
def convert_keras_to_tensorrt(keras_model_path, trt_model_dir):
    """
    Converts a Keras model to a TensorRT-optimized SavedModel.

    Args:
        keras_model_path (str): Path to the .keras model file.
        trt_model_dir (str): Directory where the TensorRT-optimized model will be saved.
    """
    print("Loading Keras model...")
    # Load the Keras model
    keras_model = tf.keras.models.load_model(keras_model_path)

    # Create a temporary directory to save the SavedModel
    temp_saved_model_dir = "temp_saved_model"
    os.makedirs(temp_saved_model_dir, exist_ok=True)

    # Save the Keras model as a SavedModel
    print("Saving Keras model as SavedModel...")
    tf.saved_model.save(keras_model, temp_saved_model_dir)

    print("Converting Keras model to TensorRT...")
    # Initialize the TensorRT converter
    params = trt.DEFAULT_TRT_CONVERSION_PARAMS._replace(
        precision_mode=trt.TrtPrecisionMode.FP16,  # Use FP16 for faster inference (if supported)
        max_workspace_size_bytes=1 << 30          # 1GB workspace size
    )
    # Pass the temporary SavedModel directory to the converter
    converter = trt.TrtGraphConverterV2(input_saved_model_dir=temp_saved_model_dir, conversion_params=params)

    # Convert the Keras model
    converter.convert()

    # Save the TensorRT-optimized model
    print(f"Saving TensorRT-optimized model to {trt_model_dir}...")
    converter.save(trt_model_dir)
    print(f"TensorRT-optimized model saved at {trt_model_dir}")

    # Optionally remove the temporary SavedModel directory
    # import shutil
    # shutil.rmtree(temp_saved_model_dir)

In [7]:
def load_tensorrt_model_and_infer(trt_model_dir, input_data):
    """
    Loads a TensorRT-optimized model and performs inference.

    Args:
        trt_model_dir (str): Directory of the TensorRT-optimized model.
        input_data (numpy.ndarray): Input data for inference.

    Returns:
        numpy.ndarray: Model predictions.
    """
    print("Loading TensorRT-optimized model...")
    trt_model = tf.saved_model.load(trt_model_dir)
    infer = trt_model.signatures["serving_default"]

    print("Running inference...")
    # Perform inference
    predictions = infer(tf.convert_to_tensor(input_data))
    return predictions

In [10]:
keras_model_path = "/content/drive/MyDrive/MCW/ASSIGNMENT-1/DataAgumentedModel.keras"         # Path to your .keras model
trt_model_dir = "/content/drive/MyDrive/MCW/ASSIGNMENT-1/model_trt"

In [9]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [11]:
convert_keras_to_tensorrt(keras_model_path, trt_model_dir)

Loading Keras model...
Saving Keras model as SavedModel...
Converting Keras model to TensorRT...
Saving TensorRT-optimized model to /content/drive/MyDrive/MCW/ASSIGNMENT-1/model_trt...
TensorRT-optimized model saved at /content/drive/MyDrive/MCW/ASSIGNMENT-1/model_trt


In [12]:
from tensorflow.keras.models import load_model

In [13]:
import tensorflow as tf
from tensorflow.keras.models import load_model

# Original model
original_model = load_model('/content/drive/MyDrive/MCW/ASSIGNMENT-1/DataAgumentedModel.keras')

# Save the original model as a SavedModel
tf.saved_model.save(original_model, '/content/original_saved_model')

# Load the original model as a SavedModel
original_saved_model = tf.saved_model.load('/content/original_saved_model')

# Now you can access the signatures
original_serving_fn = original_saved_model.signatures['serving_default']

# TensorRT-optimized model (no changes needed)
optimized_model = tf.saved_model.load('/content/drive/MyDrive/MCW/ASSIGNMENT-1/model_trt')
optimized_serving_fn = optimized_model.signatures['serving_default']

In [14]:
import numpy as np

# Sample batch of data for benchmarking
input_shape = (32, 32, 3)
batch_size = 32
dummy_input = np.random.random((batch_size, *input_shape)).astype(np.float32)
original_input = tf.convert_to_tensor(dummy_input)
optimized_input = tf.convert_to_tensor(dummy_input)

In [15]:
import time  # Import the 'time' module

def measure_latency(model_fn, input_key, input_tensor, warmup=10, iterations=100):
    """
    Measures the average latency of a TensorFlow model.

    Args:
        model_fn: The model function to be measured.
        input_key (str): The key used to pass the input tensor to the model function.
                           Should be 'inputs' for SavedModel signatures.
        input_tensor: The input tensor to be used for inference.
        warmup (int): The number of warm-up runs to perform before measuring latency.
        iterations (int): The number of iterations to run for latency measurement.

    Returns:
        float: The average latency in milliseconds.
    """

    # Warm-up runs
    for _ in range(warmup):
        _ = model_fn(**{input_key: input_tensor})  # Pass input using the correct key

    # Measure latency
    start_time = time.time()
    for _ in range(iterations):
        _ = model_fn(**{input_key: input_tensor})  # Pass input using the correct key
    end_time = time.time()

    # Calculate average latency
    avg_latency = (end_time - start_time) / iterations
    return avg_latency * 1000  # Convert to milliseconds

In [16]:
original_latency = measure_latency(
    original_serving_fn, "inputs", original_input # Changed input_key to 'inputs'
)

In [17]:
optimized_latency = measure_latency(
    optimized_serving_fn, "inputs", optimized_input
)

In [18]:
print(f"Original Model Latency: {original_latency:.2f} ms")
print(f"Optimized Model Latency: {optimized_latency:.2f} ms")

Original Model Latency: 2.57 ms
Optimized Model Latency: 0.53 ms
